In [1]:
# obtain average statistical values
import pandas as pd
import numpy as np
import os.path as pathlib

In [2]:
external_test = '/ssd/kangdang/mmc_project_2023_submission/external_test_code/fusion/external_test_fusion_data_after_iq_processed_fixed.csv'
external_test = pd.read_csv(external_test)
param_set = set(['param2', 'param3', 'param4', 'param5', 'param6', 'param7', 'param35'])

In [3]:
param_transfer_dict = dict()
param_transfer_dict['param2'] = [54.29, 11.60]
param_transfer_dict['param3'] = [77.34, 11.34]
param_transfer_dict['param4'] = [131.40, 19.06]
param_transfer_dict['param5'] = [80.826, 11.848]
param_transfer_dict['param6'] = [25.62, 3.77]
param_transfer_dict['param7'] = [6.80, 6.71]
param_transfer_dict['param35'] = [0.0, 1.0]

def get_dataset_stats(curr_csv):
    param_str = 'param35'
    curr_param_values = np.array(list(curr_csv[param_str]))
    curr_mean, curr_std = param_transfer_dict[param_str]
    curr_param_values = curr_param_values * curr_std + curr_mean
    param_str = 'param99'
    curr_csv[param_str] = curr_param_values>=1400
    
    for idx in list(range(12)) + [34, 98]:
        param_str = 'param' + str(idx + 1)
        print(param_str)
        if param_str not in param_transfer_dict.keys():
            print(curr_csv[param_str].value_counts())
            print(curr_csv[param_str].value_counts(normalize=True))
            missing_value_no = int(np.sum(curr_csv[param_str].isnull()))
            print('missing: ' + str(missing_value_no) + ', perc=' + str(round(float(missing_value_no/len(curr_csv)),3)))
        else:
            curr_param_values = np.array(list(curr_csv[param_str]))
            curr_mean, curr_std = param_transfer_dict[param_str]
            curr_param_values = curr_param_values * curr_std + curr_mean
            print('mean is : ' + str(round(np.nanmean(curr_param_values), 2)) + ', std is : ' + str(round(np.nanstd(curr_param_values), 2)))
            missing_value_no = int(np.sum(curr_csv[param_str].isnull()))
            print('missing: ' + str(missing_value_no) + ', perc=' + str(round(float(missing_value_no/len(curr_csv)),3)))
        print('\n')

In [4]:
get_dataset_stats(external_test)

param1
param1
1.0    2658
2.0    1241
Name: count, dtype: int64
param1
1.0    0.681713
2.0    0.318287
Name: proportion, dtype: float64
missing: 0, perc=0.0


param2
mean is : 52.52, std is : 11.74
missing: 0, perc=0.0


param3
mean is : 78.6, std is : 10.09
missing: 805, perc=0.206


param4
mean is : 128.73, std is : 14.76
missing: 805, perc=0.206


param5
mean is : 78.63, std is : 10.22
missing: 807, perc=0.207


param6
mean is : 25.88, std is : 4.16
missing: 100, perc=0.026


param7
mean is : 6.46, std is : 7.02
missing: 980, perc=0.251


param8
param8
0.0    1853
1.0    1158
Name: count, dtype: int64
param8
0.0    0.61541
1.0    0.38459
Name: proportion, dtype: float64
missing: 888, perc=0.228


param9
param9
0.0    2363
1.0     645
Name: count, dtype: int64
param9
0.0    0.785572
1.0    0.214428
Name: proportion, dtype: float64
missing: 891, perc=0.229


param10
param10
0.0    1854
1.0     292
Name: count, dtype: int64
param10
0.0    0.863933
1.0    0.136067
Name: proportion, dtyp

In [6]:
from glob import glob
from copy import deepcopy
numerical_param_set = ['param2', 'param3', 'param4', 'param5', 'param6', 'param7']
categorial_param_set = ['param1', 'param8', 'param9', 'param10', 'param11', 'param12']
all_csv_file_list = glob('/ssd/kangdang/mmc_project_2023_submission/external_test_code/fusion/external_test_mice_imputation_*.csv')
all_csv_data = [pd.read_csv(_) for _ in all_csv_file_list]
numerical_param_values_all = list()
for curr_param in  numerical_param_set:
    numerical_param_values_curr = list()
    for curr_csv_data in all_csv_data:
        numerical_param_values_curr.append(curr_csv_data[curr_param])
    numerical_param_values_all.append(np.array(numerical_param_values_curr))
print(np.array(numerical_param_values_all).shape)
numerical_param_values_all = np.mean(numerical_param_values_all, axis=1)
print(numerical_param_values_all.shape)

categorial_param_values_all = list()
for curr_param in categorial_param_set:
    categorial_param_values_curr = list()
    for curr_csv_data in all_csv_data:
        categorial_param_values_curr.append(curr_csv_data[curr_param])
    categorial_param_values_all.append(np.array(categorial_param_values_curr))
categorial_param_values_all = np.array(categorial_param_values_all)
print(np.unique(categorial_param_values_all))
print(categorial_param_values_all.shape)
categorial_param_values_all = categorial_param_values_all.astype(np.int64)
categorial_param_values_all = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=1, arr=categorial_param_values_all)
print(categorial_param_values_all.shape)

new_csv_data = deepcopy(all_csv_data[0])
for idx, name in enumerate(numerical_param_set):
    print(np.mean(new_csv_data[name]))
    new_csv_data.drop(name, axis = 1, inplace = True)
    new_csv_data[name] = numerical_param_values_all[idx]
    print(np.mean(new_csv_data[name]))
    print('\n')
for idx, name in enumerate(categorial_param_set):
    print(np.mean(new_csv_data[name]))
    new_csv_data.drop(name, axis = 1, inplace = True)
    new_csv_data[name] = categorial_param_values_all[idx]
    print(np.mean(new_csv_data[name]))
    print('\n')

(6, 20, 3899)
(6, 3899)
[0 1 2]
(6, 20, 3899)
(6, 3899)
-0.15235360967887426
-0.1523536096788743


0.07747068506237507
0.07906744052764385


-0.1433709022707954
-0.13669256814309505


-0.1395565029768132
-0.14982852988406956


0.07143015893078065
0.06956098012895648


-0.04109968650672074
-0.05587596014679186


1.3182867401897922
1.3182867401897922


0.3744549884585791
0.33085406514490895


0.22749422928956142
0.17183893305975892


0.11900487304437035
0.07719928186714542


0.26570915619389585
0.22518594511413184


0.14311361887663504
0.11438830469351116




In [7]:
get_dataset_stats(new_csv_data)

param1
param1
1    2658
2    1241
Name: count, dtype: int64
param1
1    0.681713
2    0.318287
Name: proportion, dtype: float64
missing: 0, perc=0.0


param2
mean is : 52.52, std is : 11.74
missing: 0, perc=0.0


param3
mean is : 78.24, std is : 9.18
missing: 0, perc=0.0


param4
mean is : 128.79, std is : 13.31
missing: 0, perc=0.0


param5
mean is : 79.05, std is : 9.3
missing: 0, perc=0.0


param6
mean is : 25.88, std is : 4.11
missing: 0, perc=0.0


param7
mean is : 6.43, std is : 6.22
missing: 0, perc=0.0


param8
param8
0    2609
1    1290
Name: count, dtype: int64
param8
0    0.669146
1    0.330854
Name: proportion, dtype: float64
missing: 0, perc=0.0


param9
param9
0    3229
1     670
Name: count, dtype: int64
param9
0    0.828161
1    0.171839
Name: proportion, dtype: float64
missing: 0, perc=0.0


param10
param10
0    3598
1     301
Name: count, dtype: int64
param10
0    0.922801
1    0.077199
Name: proportion, dtype: float64
missing: 0, perc=0.0


param11
param11
0    3021
